In [46]:
import pandas as pd

In [47]:
country = "Finland"
year = 2007
election_type = "camera"
aggregation = "district"

In [48]:
df = pd.read_csv(f"data_output/{country}/{year}_{election_type}_{aggregation}.csv.gz")
print(df.shape)
df = df[df["candidate"].str.strip() != ""]
print(df.shape)
df = df[~df["candidate"].isna()]
print(df.shape)

df.head()

(2004, 4)
(2004, 4)
(2004, 4)


,district_id,party_id,candidate,value
0,KESK - CENT,NaN,*Kiviniemi Mari ...,7385
1,KESK - CENT,NaN,Vanhanen Merja,3274
2,KESK - CENT,NaN,Peltokorpi Terhi ...,2539
3,KESK - CENT,NaN,Väätäinen Juha ...,1698
4,KESK - CENT,NaN,Nevamäki Riina ...,1115


In [49]:
df = df.groupby(["candidate", "party_id", "district_id"]).agg({"value": "sum"}).reset_index()

In [50]:
coalition = df.groupby(["district_id", "party_id"]).agg({"value": "sum", "candidate": "nunique"}).rename(columns={"candidate": "Q", "value": "N"}).reset_index()
coalition.head()

,district_id,party_id,N,Q
0,AHVENANMAAN MK. - LANDSK. ÅLAND - ÅLAND,MUUT EI PUOLUEET - ÖVRIGA - OTHERS,11168,7
1,ETELÄ-SAVO - SÖDRA SAVOLAX - SOUTH SAVO,IP,149,1
2,ETELÄ-SAVO - SÖDRA SAVOLAX - SOUTH SAVO,KD,2925,3
3,ETELÄ-SAVO - SÖDRA SAVOLAX - SOUTH SAVO,KESK - CENT,30759,14
4,ETELÄ-SAVO - SÖDRA SAVOLAX - SOUTH SAVO,KOK - SAML,15530,9


In [51]:
data = pd.merge(df, coalition, on=["district_id", "party_id"])
data["vQ/N"] = data["value"] * data["Q"] / data["N"]
data.head()

,candidate,party_id,district_id,value,N,Q,vQ/N
0,*Ahde Matti ...,SDP,KOK - SAML,4783,29295,18,2.938863
1,*Jaakonsaari Liisa ...,SDP,KOK - SAML,5003,29295,18,3.074040
2,Aho Virpi ...,SDP,KOK - SAML,1065,29295,18,0.654378
3,Haapala Antti ...,SDP,KOK - SAML,465,29295,18,0.285714
4,Haapaniemi Jarmo ...,SDP,KOK - SAML,547,29295,18,0.336098


In [52]:
data.to_csv(f"data_universality/{country}_{year}_{election_type}.csv", index=False)